### 구현 방식
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸린다.
3. (쪼갠 문서를) 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다
##### Langchain을 사용하지 않으므로 문서 parsing과 chunking을 내가 직접 해줘야한다.(1, 2번 과정)

In [ ]:
from docx import Document

document = Document("./tax.docx")
full_text = ""
for index, paragraph in enumerate(document.paragraphs):
    print(f"paragraph : {paragraph.text}")
    full_text += f"{paragraph.text}\n"
    # if index == 2: break
        

# 2. 문서를 쪼갠다.

In [71]:
# import tiktoken

# encoder = tiktoken.encoding_for_model("gpt-4o")
# encoding = encoder.encode(full_text) # text를 숫자 list로 바꾼다.
# len(encoding) # 182997의 token을 가지고 있다.

In [72]:
# decoded = encoder.decode(encoding) # 숫자 list를 다시 text로 바꾼다.
# decoded

In [73]:
# 위의 cell 2개의 내용을 담은 함수를 만들어 chunk 해보자
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text) # text를 숫자 list로 바꾼다.
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        # chunk_size: i에 더해지는 값  ex) chunk_size = 3 ==> i=0, i=3, i=6...
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)  # 숫자 list를 다시 text로 바꾼다.
        text_list.append(decoded)
    return text_list

In [74]:
chunk_list = split_text(full_text, 1500)

# 3. (쪼갠 문서를) 임베딩 -> 벡터 DB에 저장

In [75]:
import chromadb
chroma_client = chromadb.Client()

In [77]:
# collection: RDB의 Table 같은 개념
collection_name = "tax_collection3"

# 새로운 collection을 만들 때
# tax_collection = chroma_client.create_collection(collection_name)

# 이미 있는 collection을 가져올 때
tax_collection = chroma_client.get_collection(name=collection_name)

In [78]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_embedding = OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-large"
)

In [79]:
tax_collection = chroma_client.get_or_create_collection(
    collection_name,
    embedding_function=openai_embedding
)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f"{index}")
    
len(chunk_list), len(id_list)

In [ ]:
tax_collection.add(
    documents=chunk_list,
    ids=id_list
)

# 4. 질문이 있을 때 벡터 DB에서 유사도 검색

In [ ]:
query = "연봉 5천만 원인 직장인의 소득세는 얼마인가요?"
# embedding 된 텍스트에서 유사도 ㄷ검색 후 답변 return
retrieved_doc = tax_collection.query(
    query_texts=query,
    n_results=3 # n_results: 가져오는 답변의 개수
) 
retrieved_doc

In [ ]:
retrieved_doc_list = retrieved_doc["documents"] # retrieved_doc.get("documents")
retrieved_doc_list

In [ ]:
retrieved_doc_list[0]

# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달(LLM 질의)

In [85]:
# from openai import OpenAI
# client = OpenAI()
# """
#     [사용자] 2020년 월드시리즈 우승자는 누구야?
#     [어시스턴트] Los Angeles Dodgers 입니다.
#     [사용자] 그 게임은 어디서 플레이됐어?

#     - messages를 이용해 위와 같이 얘기한 history를 같이 넘겨주는 것이다.
#     - by LLM은 기본적으로 채팅을 지향하기 때문
# """

# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant"},
#         {"role": "user", "content": "Who won the world series in 2020?"},
#         {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series"},
#         {"role": "user", "content": "Where was it played?"}
#     ]
# )

In [86]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"""
                당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요
                {retrieved_doc_list[0]}
            """
        },
        {
            "role": "user",
            "content": query
        }
    ]
)

In [ ]:
response

In [ ]:
response.choices[0].message.content # 원하는 답을 찾아가는 과정이 참 길다.